In [1]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import snorkel
from snorkel import SnorkelSession
%run env.py
%run src/supervision.py
session = SnorkelSession()
classes = get_candidate_classes()

In [14]:
SPLIT_DEV, SPLIT_TEST, SPLIT_TRAIN, SPLIT_INFER

(1, 2, 0, 3)

In [30]:
from snorkel.models import Candidate
pd.Series([v[0] for v in session.query(Candidate.split).all()]).value_counts()

3    396527
0     30167
1      1756
dtype: int64

In [2]:
from snorkel.models import Candidate, GoldLabel
annot_cands = session.query(Candidate, GoldLabel).join(GoldLabel).filter(Candidate.split == SPLIT_INFER).all()
len(annot_cands)

612

In [3]:
from snorkel.models import Candidate, GoldLabel
infer_cands = session.query(Candidate).filter(Candidate.split == SPLIT_INFER).all()
len(infer_cands)

396528

In [4]:
annot_ids = set([c[0].id for c in annot_cands])
len(annot_ids)

612

In [5]:
mv_cands = [r for r in infer_cands if r.id not in annot_ids]
len(mv_cands)

395916

In [11]:
# Sanity check on counts and presence
from snorkel.models import StableLabel, GoldLabel
gcandids = [r.candidate_id for r in session.query(GoldLabel).all()]
assert not any([c.id in gcandids for c in mv_cands])
assert len(infer_cands) - len(mv_cands) == len(annot_cands)
len(infer_cands) - len(mv_cands)

612

In [20]:
c = mv_cands[0]
c

InducingCytokine(Span("b'interleukin-4'", sentence=128832, chars=[90,102], words=[17,17]), Span("b'Th2'", sentence=128832, chars=[75,77], words=[14,14]))

In [21]:
c.split

3

In [32]:
idx = pd.Series(np.arange(len(mv_cands)))
idx_train = idx.sample(n=90000, random_state=1).values
idx_infer = np.setdiff1d(idx, idx_train)
len(idx), len(idx_train), len(idx_infer)

(395916, 90000, 305916)

In [34]:
assert len(idx) == len(idx_train) + len(idx_infer)

In [35]:
assert len(idx) == len(mv_cands)

In [33]:
# Move some candidates to training split (from 3 -> 0)
for i in idx_train:
    mv_cands[i].split = 0 
session.commit()

In [36]:
# Move the rest to a new infer split (from 3 -> 2)
# * from now on, the old INFER split (3) will contain only labeled candidates and should be referred to as TEST
for i in idx_infer:
    mv_cands[i].split = 2 
session.commit()

In [37]:
# Pull new candidate distribution
from snorkel.models import Candidate
pd.Series([v[0] for v in session.query(Candidate.split).all()]).value_counts()

2    305916
0    120166
1      1756
3       612
dtype: int64